In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

file_path = r"C:\Users\bhara\mental-health-classifier\data\raw\data_to_be_cleansed.csv"
df = pd.read_csv(file_path)

# Check full structure
print(df.shape)
print(df.info())
print(df.isnull().sum())
print(df.duplicated().sum())





(5957, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5957 entries, 0 to 5956
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5957 non-null   int64 
 1   text        5607 non-null   object
 2   title       5957 non-null   object
 3   target      5957 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 186.3+ KB
None
Unnamed: 0      0
text          350
title           0
target          0
dtype: int64
0


In [13]:
import pandas as pd 
import re 
import string 
import nltk 
from nltk.corpus import stopwords 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

file_pathv= "../data/raw/d3984639-b285-4eea-8665-06e109bca630.csv"
df = pd.read_csv(file_path)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = ''.join([word for word in text.split() if word not in stop_words])
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

le= LabelEncoder()
df['label_encoded'] = le.fit_transform(df['target'])

X= df['cleaned_text']
y=df['label_encoded']

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

tfidf = TfidfVectorizer(max_features=5000)
X_train_vect =tfidf.fit_transform(X_train)
X_test_vect = tfidf.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vect, y_train)

y_pred = model.predict(X_test_vect)
print(classification_report(y_test,y_pred,target_names=[str(cls)for cls in le.classes_]))

joblib.dump(model,'../models/text_classifier.pkl')
joblib.dump(tfidf, '../models/vectorizer.pkl')
joblib.dump(le, '../models/label_encoder.pkl')

print("✅ All files saved successfully in '../models/'")
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

           0       0.67      0.14      0.23       236
           1       0.26      0.94      0.40       265
           2       0.73      0.17      0.27       211
           3       0.44      0.18      0.26       229
           4       0.73      0.09      0.16       251

    accuracy                           0.32      1192
   macro avg       0.56      0.30      0.26      1192
weighted avg       0.56      0.32      0.27      1192

✅ All files saved successfully in '../models/'
